In [1]:
import os
from IPython.display import Image as ipy_image
from IPython.display import display

#from utils.demo_util import get_shape_comp_model, get_shape_comp_opt, make_dummy_batch
#from utils.qual_util import load_bert2vqsc_model, get_lang_prob, save_mesh_as_gif
from utils.qual_util import get_lang_prob, save_mesh_as_gif
from utils.util import seed_everything
from utils.util_3d import init_mesh_renderer, sdf_to_mesh
import os
from termcolor import colored, cprint
import torch
import utils.util as util
from tqdm.notebook import tqdm
from datasets.text2shape import  Text2Shape


from datasets.text2shapepp_dataset import  Text2ShapePP


%reload_ext autoreload
%autoreload 2
torch.cuda.is_available()

True

In [2]:
class Opt():
    print("class")
    
def get_shape_comp_opt(gpu_id=0):
    opt = Opt()

    # args
    gpuid=[gpu_id]
    batch_size=32
    max_dataset_size=64

    name='new_bert_overfit'

    # default args
    opt.serial_batches = False
    opt.nThreads = 4

    # important args
    opt.dataset_mode = 'shapenet_code'
    opt.bert_cfg = "./configs/bert2vqsc.yaml"
    opt.seed = 111
    opt.isTrain = True
    opt.gpu_ids = gpuid
    opt.max_dataset_size=64
    opt.device = 'cuda:%s' % gpuid[0]
    opt.batch_size = batch_size
    opt.max_dataset_size = max_dataset_size
    opt.logs_dir="logs"
    opt.name = name
    opt.lr = 1e-4
    opt.save_latest_freq = 1000
    opt.print_freq = 1
    #utils.util.seed_everything(opt.seed)
    opt.phase = 'test'
    opt.nepochs = 300
    opt.nepochs_decay = 60
    opt.mlp_layers = 3
    opt.mlp_hidden = 1024
    return opt
opt = get_shape_comp_opt()

class


In [3]:


dataset = Text2Shape()
#opt = Opt()

dataset.initialize(opt,isTrain=False)

In [4]:
len(dataset)
# import numpy as np
# k = np.arange(0,156707)

64

In [ ]:
# k.shape
# test_indices = np.random.choice(156707, 16000, replace=False)

In [ ]:
# np.save('./datasets/text2shape_test_indices', test_indices)
# test_indices.shape

In [ ]:
# x = np.ones(156707)
# x[test_indices]=0
# train_indices = np.where(x==1)
# train_indices = train_indices[0]
#np.save('./datasets/train_indices', train_indices)
#test_indices

In [6]:
for i in tqdm(range(len(dataset))):
    dataset[i]

  0%|          | 0/64 [00:00<?, ?it/s]

In [7]:
dataset[19]["z_set_target"]

tensor([[[179, 179, 179, 179, 179, 179, 179, 179],
         [179, 179, 179, 179, 179, 179, 179, 179],
         [179, 179, 179, 179, 179, 179, 179, 179],
         [179, 179, 179, 179, 179, 179, 179, 179],
         [179, 179, 179, 179, 179, 179, 179, 179],
         [179, 179, 179, 179, 179, 179, 179, 179],
         [179, 179, 179, 179, 179, 179, 179, 179],
         [179, 179, 179, 179, 179, 179, 179, 179]],

        [[179, 179, 179, 179, 179, 179, 179, 179],
         [179, 179, 179, 179, 179, 179, 179, 179],
         [179, 179, 179, 179, 179, 179, 179, 179],
         [179, 179, 179, 179, 179, 179, 179, 179],
         [179, 179, 179, 179, 179, 179, 179, 179],
         [179, 179, 179, 179, 179, 179, 179, 179],
         [179, 179, 179, 179, 179, 179, 179, 179],
         [179, 179, 179, 179, 179, 179, 179, 179]],

        [[179, 179, 171, 272, 179, 272, 425, 179],
         [179, 179,  29, 349, 425,  29, 112, 179],
         [179, 179, 112, 272, 349, 228, 179, 179],
         [179, 179, 453,  2

In [117]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

text1 = "thin legs"
text2 = "round arms"
# tokenized1 = tokenizer(text1,return_tensors='pt',padding=True)
# x = bertmodel(**tokenized1).pooler_output
# x.shape
# tokenized2 = tokenizer(text2,return_tensors='pt',padding=True)
# x2 = bertmodel(**tokenized2).pooler_output
# x2.shape
x1 = model.encode(text1)
x2 = model.encode(text2)
x1.shape
x2.shape
model.eval()



SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

In [118]:
cos = torch.nn.CosineSimilarity(dim=0)

In [119]:
cos(torch.Tensor(x1),torch.Tensor(x2))


tensor(0.4562)

In [15]:
from models.bert2vq_scmodel_v4 import BERT2VQSCModel

In [16]:
m = BERT2VQSCModel()
m.initialize(opt)

torch.cuda.is_available()

---------- Networks initialized -------------
-----------------------------------------------


True

In [9]:
import copy
data = copy.deepcopy(dataset[4])
# data2 = dataset[2]
data["z_set_prev"] = data["z_set_prev"].unsqueeze(0)
data["z_set_target"] = data["z_set_target"].unsqueeze(0)
data["current_text"] = [data["current_text"]]


In [55]:
m.net.eval()
m.set_input(data)
m.forward()
m.outp.shape

torch.Size([1, 8, 8, 8, 512])

In [ ]:
m.backward()

In [ ]:
x = dataset[0]["z_set_prev"].shape
y = dataset[3]["z_set_prev"].shape
x

In [ ]:
x[x!=0].shape

In [10]:
from torch.utils.data import DataLoader
train_dl =  DataLoader(
            dataset,
            batch_size=opt.batch_size,
            shuffle=False,
            drop_last=True,
            num_workers=0)

In [17]:
options=opt
model = m

def train_one_epoch3(pt_profiler=None,epoch=0):
    global total_steps
    global train_loss
    global valid_loss

    epoch_iter = 0
    for i, data in enumerate(train_dl):
        iter_start_time = time.time()
        #visualizer.reset()
        #import pdb;pdb.set_trace()
        total_steps += options.batch_size
        epoch_iter += options.batch_size
        model.set_input(data)

        model.optimize_parameters(total_steps)

        nBatches_has_trained = total_steps // options.batch_size
        # if total_steps % options.print_freq == 0:

        if nBatches_has_trained % options.print_freq == 0:
            errors = model.get_current_errors()
            #import pdb;pdb.set_trace()
            #print(model.loss,"LOSS")
            t = (time.time() - iter_start_time) / options.batch_size
#             visualizer.print_current_errors(
#                 epoch, epoch_iter, total_steps, errors, t)

        # if (nBatches_has_trained % options.display_freq == 0) or i == 0:
        #     # eval
        #     model.inference(data)
        #     visualizer.display_current_results(
        #        model.get_current_visuals(), total_steps, phase='train')

        #     #model.set_input(next(test_dg))
        #     test_data = next(test_dg)
        #     model.inference(test_data.unsqueeze(0))
        #     visualizer.display_current_results(
        #        model.get_current_visuals(), total_steps, phase='test')

#         if total_steps % options.save_latest_freq == 0:
#             cprint('saving the latestmodel (epoch %d, total_steps %d)' %
#                    (epoch, total_steps), 'blue')
#             latest_name = f'epoch-latest'
#             model.save(latest_name)

        if pt_profiler is not None:
            pt_profiler.step()

In [ ]:
import time
import numpy as np
cprint('[*] Start training. name: %s' % options.name, 'blue')
total_steps = 0
train_loss = []
valid_loss = []
for epoch in tqdm(range(options.nepochs + options.nepochs_decay)):
    epoch_start_time = time.time()
    # epoch_iter = 0

    # profile
    train_one_epoch3(None,epoch)

#     if epoch % 100000 == 0:
#         cprint('saving the model at the end of epoch %d, iters %d' %
#                (epoch, total_steps), 'blue')
#         latest_name = f'epoch-latest'
#         model.save(latest_name)
#         cur_name = f'epoch-{epoch}'
#         model.save(cur_name)

    cprint(f'[*] End of epoch %d / %d \t Time Taken: %d sec \n%s' %
           (
               epoch, options.nepochs + options.nepochs_decay,
               time.time() - epoch_start_time,
               os.path.abspath(os.path.join(options.logs_dir, options.name))
           ), 'blue', attrs=['bold']
           )
    #import pdb;pdb.set_trace()
    print("Model Loss:", m.loss.item())
    model.update_learning_rate()


[*] Start training. name: new_bert_overfit


  0%|          | 0/360 [00:00<?, ?it/s]

[*] End of epoch 0 / 360 	 Time Taken: 0 sec 
/cluster/54/streakfull/ADL4CV/Project/src/plz-autosdf/logs/new_bert_overfit
Model Loss: 5.678861618041992
[*] learning rate = 0.0000100
[*] End of epoch 1 / 360 	 Time Taken: 0 sec 
/cluster/54/streakfull/ADL4CV/Project/src/plz-autosdf/logs/new_bert_overfit
Model Loss: 4.628794193267822
[*] learning rate = 0.0000200
[*] End of epoch 2 / 360 	 Time Taken: 0 sec 
/cluster/54/streakfull/ADL4CV/Project/src/plz-autosdf/logs/new_bert_overfit
Model Loss: 3.914011001586914
[*] learning rate = 0.0000300
[*] End of epoch 3 / 360 	 Time Taken: 0 sec 
/cluster/54/streakfull/ADL4CV/Project/src/plz-autosdf/logs/new_bert_overfit
Model Loss: 3.3226430416107178
[*] learning rate = 0.0000400
[*] End of epoch 4 / 360 	 Time Taken: 0 sec 
/cluster/54/streakfull/ADL4CV/Project/src/plz-autosdf/logs/new_bert_overfit
Model Loss: 2.5603318214416504
[*] learning rate = 0.0000500
[*] End of epoch 5 / 360 	 Time Taken: 0 sec 
/cluster/54/streakfull/ADL4CV/Project/src/

[*] End of epoch 45 / 360 	 Time Taken: 0 sec 
/cluster/54/streakfull/ADL4CV/Project/src/plz-autosdf/logs/new_bert_overfit
Model Loss: 0.7937575578689575
[*] learning rate = 0.0000466
[*] End of epoch 46 / 360 	 Time Taken: 0 sec 
/cluster/54/streakfull/ADL4CV/Project/src/plz-autosdf/logs/new_bert_overfit
Model Loss: 0.7832062244415283
[*] learning rate = 0.0000461
[*] End of epoch 47 / 360 	 Time Taken: 0 sec 
/cluster/54/streakfull/ADL4CV/Project/src/plz-autosdf/logs/new_bert_overfit
Model Loss: 0.7725865840911865
[*] learning rate = 0.0000456
[*] End of epoch 48 / 360 	 Time Taken: 0 sec 
/cluster/54/streakfull/ADL4CV/Project/src/plz-autosdf/logs/new_bert_overfit
Model Loss: 0.762481689453125
[*] learning rate = 0.0000452
[*] End of epoch 49 / 360 	 Time Taken: 0 sec 
/cluster/54/streakfull/ADL4CV/Project/src/plz-autosdf/logs/new_bert_overfit
Model Loss: 0.7461782693862915
[*] learning rate = 0.0000447
[*] End of epoch 50 / 360 	 Time Taken: 0 sec 
/cluster/54/streakfull/ADL4CV/Proj

[*] End of epoch 90 / 360 	 Time Taken: 0 sec 
/cluster/54/streakfull/ADL4CV/Project/src/plz-autosdf/logs/new_bert_overfit
Model Loss: 0.45652052760124207
[*] learning rate = 0.0000331
[*] End of epoch 91 / 360 	 Time Taken: 0 sec 
/cluster/54/streakfull/ADL4CV/Project/src/plz-autosdf/logs/new_bert_overfit
Model Loss: 0.46602532267570496
[*] learning rate = 0.0000330
[*] End of epoch 92 / 360 	 Time Taken: 0 sec 
/cluster/54/streakfull/ADL4CV/Project/src/plz-autosdf/logs/new_bert_overfit
Model Loss: 0.4625662565231323
[*] learning rate = 0.0000328
[*] End of epoch 93 / 360 	 Time Taken: 0 sec 
/cluster/54/streakfull/ADL4CV/Project/src/plz-autosdf/logs/new_bert_overfit
Model Loss: 0.46173807978630066
[*] learning rate = 0.0000326
[*] End of epoch 94 / 360 	 Time Taken: 0 sec 
/cluster/54/streakfull/ADL4CV/Project/src/plz-autosdf/logs/new_bert_overfit
Model Loss: 0.43536385893821716
[*] learning rate = 0.0000324
[*] End of epoch 95 / 360 	 Time Taken: 0 sec 
/cluster/54/streakfull/ADL4CV

In [127]:
x1 = dataset[0]
x2 = dataset[2]

In [74]:
x1.keys()

dict_keys(['current_text', 'z_set_prev', 'z_set_target'])

In [128]:
#x1["current_text"] = [x1["current_text"]]
x1["current_text"] = ["thin legs fafhaskh faskfhgaskh fhaskfhasf"]
x1["z_set_prev"] = x1["z_set_prev"].unsqueeze(0)
x1["z_set_target"] = x1["z_set_target"].unsqueeze(0)
model.set_input(x1)

In [129]:
model.net.eval()
model.forward()

In [97]:
model.outp.shape


torch.Size([1, 8, 8, 8, 512])

In [130]:
from einops import rearrange
k2 = x1["z_set_target"]
k2.shape
#k = rearrange(k2, 'bs d1 d2 d3 c ->bs c d1 d2 d3')

torch.Size([1, 8, 8, 8])

In [131]:
f = model.outp.cpu()
f.shape

torch.Size([1, 8, 8, 8, 512])

In [132]:
sf = torch.nn.Softmax(dim=-1)
f = sf(f)
sampler = torch.distributions.categorical.Categorical(f)
f = sampler.sample()
f.shape

torch.Size([1, 8, 8, 8])

In [110]:
cos = torch.nn.CosineSimilarity(dim=-1)
#k2.shape

In [114]:
cos(f.float().flatten(),k2.float().flatten())
#f
#f
#k2

tensor([[[[179, 179, 179, 179, 179, 179, 179, 179],
          [179, 179, 179, 179, 179, 179, 179, 179],
          [179, 179, 179, 179, 179, 179, 179, 179],
          [179, 179, 179, 179, 179, 179, 179, 179],
          [179, 179, 179, 179, 179, 179, 179, 179],
          [179, 179, 179, 179, 179, 179, 179, 179],
          [179, 179, 179, 179, 179, 179, 179, 179],
          [179, 179, 179, 179, 179, 179, 179, 179]],

         [[179, 179, 179,  68, 179, 179, 179, 179],
          [179, 179, 349, 425,  68, 112, 179, 179],
          [179,  68,  97,  97, 112, 425, 179, 179],
          [179, 425, 112, 112,  68,  68, 425, 179],
          [179,  24,  97,  68, 112, 112, 179, 179],
          [179,  68,  97, 171, 179, 112, 179, 179],
          [179, 112,  97, 453, 179, 179, 179, 179],
          [179, 179, 228, 453, 179, 179, 179, 179]],

         [[179, 179, 453, 179, 112,  29, 179, 179],
          [179, 179, 453,  97, 453,  97, 453, 179],
          [179,  24, 349,  97,  97,  97, 272, 179],
        

In [112]:
z1 = torch.rand((1,8,8,8,512))
z2 = torch.zeros((1,8,8,8,512))*-1/512
outp1 = m.net.forward(["wheels"],z1.cuda())
outp2 = m.net.forward(["okkkasfasf"],z2.cuda())

RuntimeError: Tensors must have same number of dimensions: got 5 and 6

In [ ]:
cos(outp1.flatten(),outp2.flatten())

In [26]:
k2

tensor([[[[179, 179, 179, 179, 179, 179, 179, 179],
          [179, 179, 179, 179, 179, 179, 179, 179],
          [179, 179, 179, 179, 179, 179, 179, 179],
          [179, 179, 179, 179, 179, 179, 179, 179],
          [179, 179,  24,  24,  24,  24, 179, 179],
          [179, 179, 425, 425, 425, 425, 179, 179],
          [179, 179, 179, 179, 179, 179, 179, 179],
          [179, 179, 179, 179, 179, 179, 179, 179]],

         [[179, 179, 171, 228, 228, 171, 179, 179],
          [179, 179,  97,  97,  97,  97, 179, 179],
          [179,  24,  97,  97,  97,  97, 425, 179],
          [179, 425,  97,  97,  97,  97, 453, 179],
          [179,  68,  97,  97,  97,  97, 272, 179],
          [179, 453,  97,  97,  97,  97,  29, 179],
          [179, 179,  68, 453, 112, 112,  68, 179],
          [179, 179,  24, 112, 179, 179, 179, 179]],

         [[179, 179, 425, 112, 112, 112, 179, 179],
          [179, 179, 425, 112, 112,  68, 179, 179],
          [179, 179, 112, 171, 171, 171, 272, 179],
        

In [ ]:
dataset[5]["previous_text"]

In [ ]:
dataset[18]["current_text"]

In [ ]:
import torch.nn as nn

In [ ]:
loss = nn.CrossEntropyLoss()

In [ ]:
input = torch.randn(3, 5, requires_grad=True)

In [ ]:
target = torch.empty(3, dtype=torch.long).random_(5)

In [ ]:
input.shape

In [ ]:
target.shape

In [ ]:
input = torch.randn(32,512,8,8,8, requires_grad=True)

In [ ]:
input.shape

In [ ]:
target = torch.randn(32,8,8,8, 512).softmax(dim=1)

In [ ]:
target.shape
input.shape

In [ ]:
loss(input,target)